In [20]:
import numpy as np
import pickle

import cv2
from PIL import Image
import base64

import image_pb2 as image_pb

In [21]:
im = Image.open('../../cv/000000000575.jpg')

In [22]:
def read_image(func, path='../../cv/000000000575.jpg'):
    return np.array(func(path))

In [23]:
def ppdet_read(path):
    with open(path, 'rb') as f:
        data = f.read()
    
    data = np.frombuffer(data, dtype='uint8')
    im = cv2.imdecode(data, 1)
    
    return im

In [24]:
%timeit read_image(Image.open)

8.01 ms ± 67.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
%timeit read_image(cv2.imread)

7.76 ms ± 75.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [26]:
%timeit read_image(ppdet_read)

7.28 ms ± 15.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
im_array = np.array(im)
np.save('./test.jpg.npy', im_array)

In [24]:
%timeit np.load('./test.jpg.npy')

163 µs ± 2.02 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [25]:
im_array = np.array(im)
with open('./test.jpg.pkl', 'wb') as f:
    pickle.dump(im_array, f)

In [26]:
%timeit pickle.load(open('./test.jpg.pkl', 'rb'))

48.5 µs ± 2.15 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


method | speed | size
---|--- | ---
PIL | 7.98 ms | 530K
cv2 | 7.69 ms | 530K
proto | 2.66 ms | 1.3M
numpy | 163 µs | 962K
pickle | 48.5 µs | 962K

In [5]:
im_array = np.array(im)
print(im_array.shape)

data = base64.b64encode(im_array)
shape = im_array.shape

image = image_pb.Image(data=data, shape=shape, dtype=str(im_array.dtype))
images = image_pb.Images(image=[image for _ in range(1000)])


# mean = np.frombuffer(base64.b64decode(response.data))
with open('./test.jpg.proto', 'wb') as f:
    f.write(image.SerializeToString())

# with open('./tests.jpg.proto', 'wb') as f:
#     f.write(images.SerializeToString())


(513, 640, 3)


In [4]:
def parse_proto(path):
    image = image_pb.Image()
    image.ParseFromString(open(path, 'rb').read())
    
    data = np.frombuffer(base64.b64decode(image.data), dtype=image.dtype)
    data = data.reshape(*image.shape)
    
    return data

In [5]:
%timeit parse_proto('./test.jpg.proto')

2.47 ms ± 41 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
# protov1

In [4]:
im_array = np.array(im)
print(im_array.shape)

shape = im_array.shape
im_list = im_array.reshape(-1).tolist()

# data = base64.b64encode(im_array)

image = image_pb.ImageV1(data=im_list, shape=shape, dtype=str(im_array.dtype))
# images = image_pb.Images(image=[image for _ in range(1000)])

# mean = np.frombuffer(base64.b64decode(response.data))
with open('./test.jpg.v1.proto', 'wb') as f:
    f.write(image.SerializeToString())

# with open('./tests.jpg.proto', 'wb') as f:
#     f.write(images.SerializeToString())


(513, 640, 3)


In [10]:
def parse_proto_v1(path):
    image = image_pb.ImageV1()
    image.ParseFromString(open(path, 'rb').read())
    
    # data = np.frombuffer(base64.b64decode(image.data), dtype=image.dtype)
    # data = data.reshape(*image.shape)
    data = np.array(image.data, dtype=image.dtype).reshape(*image.shape)
    
    return data

In [11]:
%timeit parse_proto_v1('./test.jpg.v1.proto')

74.4 ms ± 801 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
